In [1]:
import sqlite3
import pandas as pd
from datetime import datetime, timedelta

In [2]:
# Create a new SQLite database (or connect to an existing one)
conn = sqlite3.connect(':memory:')  # Using in-memory database for demonstration
cursor = conn.cursor()

In [3]:
# Create sample tables
cursor.execute('''
CREATE TABLE employees (
    employee_id INTEGER PRIMARY KEY,
    name TEXT,
    department_id INTEGER,
    hire_date DATE
)
''')

In [4]:
cursor.execute('''
CREATE TABLE departments (
    department_id INTEGER PRIMARY KEY,
    department_name TEXT
)
''')

In [5]:

cursor.execute('''
CREATE TABLE salaries (
    salary_id INTEGER PRIMARY KEY,
    employee_id INTEGER,
    salary_amount DECIMAL,
    salary_date DATE,
    FOREIGN KEY (employee_id) REFERENCES employees(employee_id)
)
''')

In [6]:
# Insert sample data
employees_data = [
    (1, 'Alice', 1, '2020-01-15'),
    (2, 'Bob', 2, '2019-03-22'),
    (3, 'Charlie', 1, '2021-05-13')
]

departments_data = [
    (1, 'HR'),
    (2, 'Engineering')
]

salaries_data = [
    (1, 1, 60000, '2023-01-01'),
    (2, 2, 80000, '2023-01-01'),
    (3, 3, 55000, '2023-01-01')
]

cursor.executemany('INSERT INTO employees VALUES (?, ?, ?, ?)', employees_data)
cursor.executemany('INSERT INTO departments VALUES (?, ?)', departments_data)
cursor.executemany('INSERT INTO salaries VALUES (?, ?, ?, ?)', salaries_data)

conn.commit()

In [7]:
# Query with complex joins, subqueries, and date functions
query = '''
SELECT
    e.name AS employee_name,
    d.department_name,
    s.salary_amount,
    e.hire_date,
    (julianday('now') - julianday(e.hire_date)) / 365 AS years_with_company
FROM
    employees e
    INNER JOIN departments d ON e.department_id = d.department_id
    LEFT JOIN salaries s ON e.employee_id = s.employee_id
WHERE
    s.salary_date = (
        SELECT
            MAX(salary_date)
        FROM
            salaries
        WHERE
            employee_id = e.employee_id
    )
ORDER BY
    years_with_company DESC
'''

In [8]:
# Execute the query and fetch the results
results = pd.read_sql_query(query, conn)

In [9]:
# Display the results
print(results)

  employee_name department_name  salary_amount   hire_date  years_with_company
0           Bob     Engineering          80000  2019-03-22            5.253317
1         Alice              HR          60000  2020-01-15            4.434139
2       Charlie              HR          55000  2021-05-13            3.108111


In [10]:
# Close the connection
conn.close()